# wo shi ji ji kun !!!
# wuda shi wo ge!!!

In [1]:
from lmfit import Parameters
from lmfit import minimize
import numpy as np
import pandas as pd

### Define parameters

In [2]:
# risk aversion
delta = 4  # 3.5 < delta < 4.5

In [3]:
# convariance_quantities
sigma_all = [[.02778, .00387, .00021],
             [.00387, .01112, -.00020],
             [.00021, -.00020, .00115],
            ]

In [4]:
pd.DataFrame(sigma_all)

,0,1,2
0,0.02778,0.00387,0.00021
1,0.00387,0.01112,-0.00020
2,0.00021,-0.00020,0.00115


In [5]:
# expect_return
expect_return = [0.1073, 0.0737, 0.0627]

### Generate intial guess for xi 

In [68]:
# Generate random xi or input yours as list
nbr_assets = len(expect_return)
xi_weights = list(np.random.dirichlet(np.ones(nbr_assets),size=1)[0]) # generate random xi list
if sum(xi_weights)!=1: # test the sum is 1
    raise ValueError("The sum of your input weight '{}' is not '1'.".format(xi_weights))
print(xi_weights)
print("Sum: {}".format(sum(xi_weights)))

[0.5614786416208134, 0.11007931644116234, 0.32844204193802434]
Sum: 1.0


### Form params for minimization

In [84]:
# pack parameters for fitting
params = Parameters()
for n in range(len(xi_weights)):
    params.add('x' + str(n+1),
                value=xi_weights[n],
                vary=True,
                min=0,
                max=1)
print("\n----------Before apply constraint---------- \n")
params.pretty_print()

# form constraint
_param_list = list(params.valuesdict().keys())
_constraint_param = _param_list.pop(-1)
_constraint = '-'.join(_param_list)
_constraint = '1-' + _constraint

# set constraint
params[_constraint_param].set(expr=_constraint)
print("\n----------After apply constraint---------- \n")
params.pretty_print()



----------Before apply constraint---------- 

Name     Value      Min      Max   Stderr     Vary     Expr Brute_Step
x1    0.5615        0        1     None     True     None     None
x2    0.1101        0        1     None     True     None     None
x3    0.3284        0        1     None     True     None     None

----------After apply constraint---------- 

Name     Value      Min      Max   Stderr     Vary     Expr Brute_Step
x1    0.5615        0        1     None     True     None     None
x2    0.1101        0        1     None     True     None     None
x3    0.3284        0        1     None    False  1-x1-x2     None


# Define model

In [87]:
def model(params, expect_return, sigma_all):
    # unpack params
    parvals = params.valuesdict()
    param_names = parvals.keys()
    xi_weights = []
    for each in param_names:
        xi_weights.append(parvals[each])
    print("Trying: {}".format(xi_weights))
    
    # calculate the cost
    sum_1 = 0
    sum_2 = 0
    for i in range(len(xi_weights)):
        j=0
        _sum_1 = xi_weights[i] * expect_return[i]
        sum_1 = sum_1 + _sum_1
#         print(i)
        while j <= len(xi_weights)-1:
#             print(i, j)
            _sum_2 = sigma_all[i][j] * xi_weights[i] * xi_weights[j]
            sum_2 = sum_2 + _sum_2
            j=j+1
    cost = sum_1 - (delta/2)*sum_2
    return cost
    

In [88]:
model(params, expect_return=expect_return, sigma_all=sigma_all)

Trying: [0.5614786416208134, 0.11007931644116234, 0.32844204193802429]


0.069836709306891634

In [52]:
def jacobian(x):
    return np.array((-2*.5*(1 - x[0]) - 4*x[0]*(x[1] - x[0]**2), 2*(x[1] - x[0]**2)))

In [89]:
result = minimize(model, params, method='newton', jac=jacobian, args=(expect_return, sigma_all))

Trying: [0.5614786416208134, 0.11007931644116231, 0.32844204193802429]
Trying: [0.66944115806991489, 0.52531769711648824, 0]
Trying: [0.59546342761710191, 0.21686493325310963, 0.18767163912978846]
Trying: [0.5956594399581987, 0.21752880707810096, 0.18681175296370034]
Trying: [0.59644333642366509, 0.2201916073017845, 0.18336505627455041]
Trying: [0.59957644090214191, 0.2309577464212636, 0.1694658126765945]
Trying: [0.61206692050960343, 0.2757343756852661, 0.11219870380513047]
Trying: [0.6123024394006521, 0.27646070172379444, 0.11123685887555346]
Trying: [0.6132442520331417, 0.27937188699004278, 0.10738386097681552]
Trying: [0.61700723406967484, 0.29110856789118522, 0.091884198039139942]
Trying: [0.63198693456404742, 0.33938601426513315, 0.028627051170819429]
Trying: [0.63252612570794675, 0.34076016703436385, 0.026713707257689401]
Trying: [0.63468122644462921, 0.34627006074658034, 0.019048712808790447]
Trying: [0.6432742543350819, 0.36850913184334549, 0]
Trying: [0.64329212658676216, 0.3

In [90]:
result.params

Parameters([('x1', <Parameter 'x1', 0.6432742543350819, bounds=[0:1]>),
            ('x2', <Parameter 'x2', 0.36850913184334549, bounds=[0:1]>),
            ('x3', <Parameter 'x3', 0, bounds=[0:1], expr='1-x1-x2'>)])

In [91]:
result.__dict__

{'aborted': False,
 'aic': -1.4210500220496325,
 'bic': -5.4210500220496325,
 'chisqr': 0.0044225004859474292,
 'errorbars': False,
 'fun': 0.06650188332631964,
 'init_vals': [0.12326923170405091, -0.8944123608221225],
 'init_values': {'x1': 0.12326923170405091, 'x2': -0.8944123608221225},
 'jac': array([-0.30184059, -0.70114553]),
 'message': 'Warning: Desired error not necessarily achieved due to precision loss.',
 'method': 'Newton-CG',
 'ndata': 1,
 'nfev': 35,
 'nfree': -1,
 'nhev': 0,
 'nit': 4,
 'njev': 36,
 'nvarys': 2,
 'params': Parameters([('x1',
              <Parameter 'x1', 0.6432742543350819, bounds=[0:1]>),
             ('x2', <Parameter 'x2', 0.36850913184334549, bounds=[0:1]>),
             ('x3', <Parameter 'x3', 0, bounds=[0:1], expr='1-x1-x2'>)]),
 'redchi': 0.0044225004859474292,
 'residual': array([ 0.06650188]),
 'status': 2,
 'success': False,
 'var_names': ['x1', 'x2'],
 'x': array([ 0.29062232, -0.26611143])}